In [1]:
import ROOT

Welcome to JupyROOT 6.20/04


pen the rootfile and get the workspace from the exercise_0

In [2]:
fInput = ROOT.TFile("Workspace_mumufit.root")
ws = fInput.Get("ws")
ws.Print()


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt


RooWorkspace(ws) ws contents

variables
---------
(NJpsi,Nbkg,a1,a2,a3,alphaJpsi,cross_psi,eff_psi,lumi_psi,mass,meanJpsi,meanpsi2S,nJpsi,sigmaJpsi)

p.d.f.s
-------
RooCBShape::CBJpsi[ m=mass m0=meanJpsi sigma=sigmaJpsi alpha=alphaJpsi n=nJpsi ] = 0.012173
RooCBShape::CBpsi2S[ m=mass m0=meanpsi2S sigma=sigmaJpsi alpha=alphaJpsi n=nJpsi ] = 0.00289326
RooChebychev::backgroundPDF[ x=mass coefList=(a1,a2,a3) ] = 1.35122
RooAddPdf::totPDF[ NJpsi * CBJpsi + Npsi * CBpsi2S + Nbkg * backgroundPDF ] = 0.91405

functions
--------
RooFormulaVar::Npsi[ actualVars=(eff_psi,lumi_psi,cross_psi) formula="x[0]*x[1]*x[2]" ] = 5.72535

datasets
--------
RooDataSet::data(mass)



ou can set constant parameters that are known<br>
f you leave them floating, the fit procedure will determine their uncertainty

In [3]:
ws.var("meanJpsi").setConstant(1)

et the RooModelConfig and let it know what the content of the workspace is about

In [4]:
model = ROOT.RooStats.ModelConfig()
model.SetWorkspace(ws)
model.SetPdf("totPDF")

ere we explicitly set the value of the parameters for the null hypothesis<br>
e want no signal contribution, so cross_psi = 0

In [5]:
cross_psi = ws.var("cross_psi")
poi = ROOT.RooArgSet(cross_psi)
nullParams = poi.snapshot()
nullParams.setRealValue("cross_psi",0.)

False

uild the profile likelihood calculator

In [7]:
plc = ROOT.RooStats.ProfileLikelihoodCalculator(ws.data("data"), model)
plc.SetParameters(poi)
plc.SetNullParameters(nullParams)

e get a HypoTestResult out of the calculator, and we can query it.

In [8]:
htr = plc.GetHypoTest()

[#1] INFO:Minization -- p.d.f. provides expected number of events, including extended term in likelihood.
[#1] INFO:Minization -- createNLL: caching constraint set under name CONSTR_OF_PDF_totPDF_FOR_OBS_mass with 0 entries
[#0] PROGRESS:Minization -- ProfileLikelihoodCalcultor::DoGLobalFit - find MLE 
[#0] PROGRESS:Minization -- ProfileLikelihoodCalcultor::DoMinimizeNLL - using Minuit / Migrad with strategy 1
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (CBJpsi,CBpsi2S,backgroundPDF)
[#1] INFO:Minization -- 
  RooFitResult: minimized FCN value: -2191.54, estimated distance to minimum: 7.27643e-06
                covariance matrix quality: Full, accurate covariance matrix
                Status : MINIMIZE=0 

    Floating Parameter    FinalValue +/-  Error   
  --------------------  --------------------------
                 NJpsi    1.5763e+02 +/-  2.00e

In [9]:
print ("-------------------------------------------------")
print ("The p-value for the null is ", htr.NullPValue())
print ("Corresponding to a signifcance of ", htr.Significance())
print ("-------------------------------------------------")

-------------------------------------------------
The p-value for the null is  0.08749068033623891
Corresponding to a signifcance of  1.3563703547652992
-------------------------------------------------


yROOT sometimes fails cleaning memory, this helps

In [10]:
del plc